In [ ]:
from typing import List
from datetime import datetime, UTC

from binance_extras.manager import BinanceManager
from binance_extras.searchs import SearchKLines
from general_utils.coins import SymbolPairsEnum, IntervalKLineEnum, iter_all_symbol_interval
from general_utils.time_utc import get_datetime_now, get_datetime
from cripto_db.db_klines import DBKLines, CollectionKLines
from cripto_trading.kline import KLine

bm = BinanceManager()
db_klines = DBKLines()

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from general_utils.coins import SymbolPairsEnum, IntervalKLineEnum, get_collection_name

def insert_klines_concurrently(coll_kline: CollectionKLines, klines: List[KLine]):
    coll_kline.insert_klines(klines)

EXCLUDE = [
    get_collection_name(SymbolPairsEnum.BTCUSDT, IntervalKLineEnum.interval_1m),
    get_collection_name(SymbolPairsEnum.BTCUSDT, IntervalKLineEnum.interval_5m),
    get_collection_name(SymbolPairsEnum.BTCUSDT, IntervalKLineEnum.interval_1h),
    get_collection_name(SymbolPairsEnum.BTCUSDT, IntervalKLineEnum.interval_1d),

]
MAX_WORKERS = 100
for symbol, interval in iter_all_symbol_interval(EXCLUDE):
    coll_klines = db_klines.get_collection(symbol, interval)

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
        iter_futures = (pool.submit(insert_klines_concurrently, coll_klines, klines)
                        for klines in bm.iter_all_mark_price_klines(symbol, interval))
        for future in as_completed(iter_futures):
            future.result()

In [ ]:

date_i = get_datetime(2018)
date_f = get_datetime_now()
klines_total = []
for klines in bm.iter_mark_price_klines(BTCUSDT, INTERVAL_1M, 1500, date_i, date_f):
    klines_total.extend(klines)

In [ ]:
from general_utils.time_utc import timestamp2datetime
timestamp2datetime(1707004799999)

In [ ]:
from general_utils.time_utc import get_timestamp_now, get_datetime_now
date_now = get_datetime_now()
s = SearchKLines(
    symbol = SymbolPairsEnum.BTCUSDT,
    interval = IntervalKLineEnum.interval_1m,
    limit = 2,
    start_time = 1707075300000#date_now - timedelta(minutes=5),
)
klines = bm.mark_price_klines(s, False)
#klines[-1].model_dump()
klines

In [ ]:
import time

t_last = datetime.fromtimestamp(klines[-1].time_close/1000, tz=UTC)
while True:
    t_now = datetime.now(tz=UTC)
    if not(t_last > t_now):
        display(t_last)
        display(t_now)
        print("-"*20)
        time.sleep(1)
